In [4]:
import requests
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import csv
import numpy as np



In [2]:
#reads in the csv
csv_file = "../data/summer.csv"
directory_db = pd.read_csv(csv_file)

directory_db

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver
...,...,...,...,...,...,...,...,...,...
31160,2012,London,Wrestling,Wrestling Freestyle,"JANIKOWSKI, Damian",POL,Men,Wg 84 KG,Bronze
31161,2012,London,Wrestling,Wrestling Freestyle,"REZAEI, Ghasem Gholamreza",IRI,Men,Wg 96 KG,Gold
31162,2012,London,Wrestling,Wrestling Freestyle,"TOTROV, Rustam",RUS,Men,Wg 96 KG,Silver
31163,2012,London,Wrestling,Wrestling Freestyle,"ALEKSANYAN, Artur",ARM,Men,Wg 96 KG,Bronze


In [40]:
#groups country by Country and Year
country_medal_db=directory_db.groupby(['Country','Year','City']).count()
country_medal_db=country_medal_db[['Medal']]
country_medal_db.reset_index(inplace=True)

#rearranges by year
country_medal_db.sort_values('Year', ascending=True,inplace=True)
country_medal_db.set_index(['Year','Country'],inplace=True)


country_medal_db

City  Medal
Year Country               
1896 AUT      Athens      5
     GBR      Athens      7
     GER      Athens     33
     DEN      Athens      6
     GRE      Athens     52
...              ...    ...
2012 GAB      London      1
     FRA      London     82
     RUS      London    130
     GER      London     94
     QAT      London      2

[1157 rows x 2 columns]

In [41]:
#calculates Medal accumalitve sum - though I didn't end up using it
country_medal_db['Medal_Sum'] = country_medal_db.groupby('Country').cumsum()
country_medal_db

City  Medal  Medal_Sum
Year Country                          
1896 AUT      Athens      5          5
     GBR      Athens      7          7
     GER      Athens     33         33
     DEN      Athens      6          6
     GRE      Athens     52         52
...              ...    ...        ...
2012 GAB      London      1          1
     FRA      London     82       1396
     RUS      London    130        768
     GER      London     94       1305
     QAT      London      2          4

[1157 rows x 3 columns]

In [42]:
#exports csv
country_medal_db.to_csv('../Data/summer_country_medals.csv',index=True)

In [35]:
#groups country by Country and Year
olympic_summer_db=directory_db.groupby(['City','Year']).count()

#country_medal_db=country_medal_db[['Medal']]
olympic_summer_db.reset_index(inplace=True)
olympic_summer_db=olympic_summer_db[['City','Year']]
#rearranges by year
olympic_summer_db.sort_values('Year', ascending=True,inplace=True)
olympic_summer_db.set_index(['Year'],inplace=True)


olympic_summer_db.to_csv('../Data/summer_game_cities.csv',index=True)
olympic_summer_db

,City
Year,
1896,Athens
1900,Paris
1904,St Louis
1908,London
1912,Stockholm
1920,Antwerp
1924,Paris
1928,Amsterdam
1932,Los Angeles


------------------------------Start SQL File Here-----------------------------

In [5]:
database_path= "../data/Olympics.sqlite"

In [6]:
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [8]:
Base.prepare(engine, reflect=True)

In [9]:
inspector = inspect(conn)

In [10]:
inspector.get_table_names()

['countryData', 'hostsData', 'olympicsData']

In [45]:
pd.read_sql("SELECT * FROM olympicsData",conn).head(5)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,15,Arvo Ossian Aaltonen,M,30,NA,NA,Finland,FIN,1920 Summer,1920,Summer,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,Bronze
1,15,Arvo Ossian Aaltonen,M,30,NA,NA,Finland,FIN,1920 Summer,1920,Summer,Antwerpen,Swimming,Swimming Men's 400 metres Breaststroke,Bronze
2,16,Juhamatti Tapio Aaltonen,M,28,184,85,Finland,FIN,2014 Winter,2014,Winter,Sochi,Ice Hockey,Ice Hockey Men's Ice Hockey,Bronze
3,17,Paavo Johannes Aaltonen,M,28,175,64,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze
4,17,Paavo Johannes Aaltonen,M,32,175,64,Finland,FIN,1952 Summer,1952,Summer,Helsinki,Gymnastics,Gymnastics Men's Team All-Around,Bronze


In [58]:
pd.read_sql("SELECT NOC FROM olympicsData",conn)

,NOC
0,FIN
1,FIN
2,FIN
3,FIN
4,FIN
...,...
39778,NED
39779,NED
39780,POL
39781,URS


In [59]:
pd.read_sql("SELECT CountryCode FROM countryData",conn)

,CountryCode
0,ARG
1,ARG
2,ARG
3,ARG
4,ARG
...,...
1330,None
1331,None
1332,None
1333,None


In [44]:
pd.read_sql("SELECT * FROM countryData",conn).head(5)

,CountryName,CountryCode,SeriesName,SeriesCode,1960[YR1960],1962[YR1962],1964[YR1964],1966[YR1966],1968[YR1968],1970[YR1970],...,1998[YR1998],2000[YR2000],2002[YR2002],2004[YR2004],2006[YR2006],2008[YR2008],2010[YR2010],2012[YR2012],2014[YR2014],2016[YR2016]
0,Argentina,ARG,GDP (current US$),NY.GDP.MKTP.CD,..,..,..,..,..,..,...,298948250000,284203750000,9.7724e+10,1.64658e+11,2.32557e+11,3.61558e+11,4.23627e+11,5.45982e+11,5.2632e+11,5.57531e+11
1,Argentina,ARG,GDP per capita (current US$),NY.GDP.PCAP.CD,..,..,..,..,..,..,...,8289.51,7708.1,2593.4,4277.72,5919.01,9020.87,10386,13082.7,12334.8,12790.2
2,Argentina,ARG,Population growth (annual %),SP.POP.GROW,..,1.60007942669871,1.54925797693851,1.49771883580732,1.47575331677022,1.53152765510278,...,1.13222,1.10061,1.08356,1.05539,1.01545,0.99257,0.752218,1.13691,1.09946,1.05718
3,Argentina,ARG,Population density (people per sq. km of land ...,EN.POP.DNST,..,7.72942569308179,7.97475307762297,8.21942967599546,8.46579919537836,8.72607566074345,...,13.1778,13.4728,13.7691,14.0652,14.3567,14.6455,14.9043,15.2495,15.5916,15.9281
4,Argentina,ARG,"Population, total",SP.POP.TOTL,20481781,21153042,21824427,22494031,23168268,23880564,...,36063451,36870796,37681743,38491970,39289876,40080159,40788453,41733271,42669500,43590368


In [47]:
data=pd.read_sql("SELECT c.CountryName,c.CountryCode,o.NOC,o.Year,o.Season,o.City, "\
                 +"o.Sport,o.Medal,o.Sex,o.Name "
                 +"FROM olympicsData AS o JOIN countryData AS c ON "\
                 +"c.CountryCode=o.NOC",conn)
data.head(10)

,CountryName,CountryCode,NOC,Year,Season,City,Sport,Medal,Sex,Name
0,Finland,FIN,FIN,1920,Summer,Antwerpen,Swimming,Bronze,M,Arvo Ossian Aaltonen
1,Finland,FIN,FIN,1920,Summer,Antwerpen,Swimming,Bronze,M,Arvo Ossian Aaltonen
2,Finland,FIN,FIN,1920,Summer,Antwerpen,Swimming,Bronze,M,Arvo Ossian Aaltonen
3,Finland,FIN,FIN,1920,Summer,Antwerpen,Swimming,Bronze,M,Arvo Ossian Aaltonen
4,Finland,FIN,FIN,1920,Summer,Antwerpen,Swimming,Bronze,M,Arvo Ossian Aaltonen
5,Finland,FIN,FIN,1920,Summer,Antwerpen,Swimming,Bronze,M,Arvo Ossian Aaltonen
6,Finland,FIN,FIN,1920,Summer,Antwerpen,Swimming,Bronze,M,Arvo Ossian Aaltonen
7,Finland,FIN,FIN,1920,Summer,Antwerpen,Swimming,Bronze,M,Arvo Ossian Aaltonen
8,Finland,FIN,FIN,1920,Summer,Antwerpen,Swimming,Bronze,M,Arvo Ossian Aaltonen
9,Finland,FIN,FIN,1920,Summer,Antwerpen,Swimming,Bronze,M,Arvo Ossian Aaltonen


In [48]:
my_data=data.groupby(["NOC","CountryCode","CountryName","Year","Season","City","Sport","Medal","Sex"]).count()
my_data

Name
NOC CountryCode CountryName   Year Season City           Sport     Medal  Sex      
AFG AFG         Afghanistan   2008 Summer Beijing        Taekwondo Bronze M       5
                              2012 Summer London         Taekwondo Bronze M       5
ARG ARG         Argentina     1924 Summer Paris          Athletics Silver M       5
                                                         Boxing    Bronze M      10
                                                                   Silver M      10
...                                                                             ...
VEN VEN         Venezuela, RB 2008 Summer Beijing        Taekwondo Bronze F       5
                              2012 Summer London         Fencing   Gold   M       5
                              2016 Summer Rio de Janeiro Athletics Silver F       5
                                                         Boxing    Bronze M       5
                                                         Cycling   Bronze F       5

[9182 rows x 1 columns]

In [50]:
my_data=my_data[['Name']]
my_data.reset_index(inplace=True)

In [51]:
my_data.sort_values(['Year','NOC'], ascending=(True,True),inplace=True)
my_data

,NOC,CountryCode,CountryName,Year,Season,City,Sport,Medal,Sex,Name
82,AUS,AUS,Australia,1896,Summer,Athina,Athletics,Gold,M,10
83,AUS,AUS,Australia,1896,Summer,Athina,Tennis,Bronze,M,5
444,AUT,AUT,Austria,1896,Summer,Athina,Cycling,Bronze,M,10
445,AUT,AUT,Austria,1896,Summer,Athina,Cycling,Gold,M,5
446,AUT,AUT,Austria,1896,Summer,Athina,Swimming,Gold,M,5
...,...,...,...,...,...,...,...,...,...,...
9166,UZB,UZB,Uzbekistan,2016,Summer,Rio de Janeiro,Weightlifting,Gold,M,5
9167,UZB,UZB,Uzbekistan,2016,Summer,Rio de Janeiro,Wrestling,Bronze,M,15
9179,VEN,VEN,"Venezuela, RB",2016,Summer,Rio de Janeiro,Athletics,Silver,F,5
9180,VEN,VEN,"Venezuela, RB",2016,Summer,Rio de Janeiro,Boxing,Bronze,M,5


In [52]:
my_data.rename(columns={"Name":"Medals_Won"},inplace=True)

In [55]:
my_data.CountryCode.unique()

array(['AUS', 'AUT', 'FRA', 'GBR', 'HUN', 'USA', 'BEL', 'CAN', 'COL',
       'CUB', 'ESP', 'IND', 'ITA', 'LUX', 'MEX', 'NOR', 'NZL', 'SWE',
       'FIN', 'RUS', 'BRA', 'EST', 'JPN', 'ARG', 'IRL', 'POL', 'ROU',
       'CHI', 'EGY', 'TUR', 'JAM', 'KOR', 'PAN', 'PER', 'TTO', 'VEN',
       'ISL', 'PAK', 'ETH', 'GHA', 'IRQ', 'MAR', 'SGP', 'KEN', 'PRK',
       'TUN', 'CMR', 'UGA', 'LIE', 'THA', 'GUY', 'CHN', 'CIV', 'DOM',
       'SYR', 'DJI', 'SEN', 'SUR', 'ISR', 'LTU', 'NAM', 'QAT', 'BLR',
       'KAZ', 'UKR', 'UZB', 'ARM', 'AZE', 'BDI', 'CZE', 'ECU', 'GEO',
       'HKG', 'MDA', 'MOZ', 'SVK', 'KGZ', 'MKD', 'ERI', 'AFG', 'SRB',
       'TJK', 'BRN', 'CYP', 'GAB', 'MNE', 'JOR'], dtype=object)